# LeNet 1998 Model (End-to-End Trained) 

In this notebook, we will be creating a minimal baseline for our study by using the [1998 LeNet model](https://en.wikipedia.org/wiki/LeNet) as an example of a classical shallow convolutional neural network (CNN).

# Setup:

## Connect to Remote Compute Environment

First ensure we are connected to the correct VSCode Remote Kernel. This step is important, because in order to perform the k-fold cross-validation, we need fairly powerful compute resources. 

In [ ]:
!uname -nv && ls /

## Upgrade Python Modules

Install the latest version of Tensorflow, and install Tensorflow. This may take some time.

In [ ]:
!pip3 install --quiet --upgrade tensorflow==2.11.0
!pip3 install --quiet tensorflow_addons

Note: If you had to upgrade tensorflow to `2.11.0`, then you must restart your Jupyter notebook kernel in order for the latest version of tensorflow to be used.

## Python Environment Checks

Instantiate Python Kernel and load Python modules.

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import keras

from keras import layers
from typing import Literal, Union

# Import utility functions defined in ../common/ package
import sys
sys.path.append('../')
from common import *

Double-check GPU is available.

In [ ]:
display(tf.__version__)
display(tf.config.list_physical_devices('GPU'))
display(tf.test.gpu_device_name())
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], enable=True)

# Model Preparation

Begin preparing the model's execution environment. First, we start by defining some constants:



In [ ]:
BATCH_SIZE: int = 128
EPOCHS    : int = 50
IMG_SIZE  : tuple[int, int] = (299, 299)
AUTOTUNE  : Literal = tf.data.AUTOTUNE
RNG_SEED  : int = 1337

# For Remote
dataset_directory: str = "./"

# For Local
# dataset_directory: str = "../../dataset/"

## Prepare Datasets

Train, Test, and Validation dataset split:

* Total: 2936 (100%)
    * Training and Validation Set: 2490 (85%):
        * K-Fold Cross-Validation, K = 10:
            * Training Set:  2241 (~76%)
            * Validation Set: 249 (~8.5% per fold)
    * Hold-out Test Set:  441 (15%)


In [ ]:
# InceptionV3 requires image tensors with a shape of (299, 299, 3) 
ds_train: tf.data.Dataset = tf.data.Dataset.load(dataset_directory + "ds_train")
ds_valid: tf.data.Dataset = tf.data.Dataset.load(dataset_directory + "ds_valid")
ds_test : tf.data.Dataset = tf.data.Dataset.load(dataset_directory + "ds_test")

# For K-Fold Cross Validation
ds_train_and_valid: tf.data.Dataset = ds_train.concatenate(ds_test)

In [ ]:
# Batching, caching, and performance optimisations are *not* performed at this stage
# Since we are doing K-Fold validation

# configure_for_performance(ds_train)
# configure_for_performance(ds_valid)
# configure_for_performance(ds_test)

In [ ]:
preview_dataset(ds_train_and_valid)

# Define Model

In [ ]:
class LeNet1998(tf.keras.Model):
    def __init__(self, name=None, dropout_rate: float = None, weights: str = None, **kwargs):
        super().__init__(**kwargs)

        # First, we will define the different components of the model separately
        self.input_layer: tf.Tensor = layers.InputLayer(input_shape=(299, 299, 3), name="Input_Layer")
        self.data_augmentation: tf.keras.Sequential = tf.keras.Sequential(
            [
                layers.RandomFlip(seed=RNG_SEED),
            ],
            name="Data_Augmentation_Pipeline"
        )
        self.lenet1999: tf.keras.Model = tf.keras.Sequential(
            [
                layers.Conv2D(6, kernel_size=5, strides=1,  activation='tanh', padding='same'),
                layers.AveragePooling2D(),
                layers.Conv2D(16, kernel_size=5, strides=1, activation='tanh', padding='valid'),
                layers.AveragePooling2D(),
            ],
            name="Lenet1998_Model"
        )
        # In this end-to-end training baseline we train the weights ourselves
        self.classifier: tf.keras.Sequential = tf.keras.Sequential(
            [
                layers.Flatten(),
                layers.Dense(1024, activation='relu'),
                layers.Dense(18, activation='sigmoid')
            ],
            name="RUST_Score_Classifier"
        )

        # Finally, we define the model as the sum of it's components
        self.model: tf.keras.Sequential = tf.keras.Sequential(
            [
                self.input_layer,
                self.data_augmentation,
                self.lenet1999,
                self.classifier
            ],
            name="InceptionV3_End2End"
        )
    def call(self, inputs):
        return self.model(inputs)

# Train Model with K-Fold Cross-Validation

In [ ]:
kfold_history: list[dict[str, float]] = cross_validate(
    LeNet1998,
    ds_train_and_valid,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    k=10
)

In [ ]:
plot_kfold_history(kfold_history)

In [ ]:
import pickle

with open('kfold_history_lenet1998.pickle', 'wb') as handle:
    pickle.dump(kfold_history, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('kfold_history_inceptionv3_radimagenet.pickle', 'rb') as handle:
#     b = pickle.load(handle)